## Regression Tables via `pf.etable()` and the `Stargazer`

To produce regression tables, we have two options: pyfixest's internal `etable()` function and the [Stargazer](https://github.com/StatsReporting/stargazer) Python package. The following examples illustrate how to use both options.

Contents: 

-   [Regression Tables via `pf.etable()`](#regression-tables-via-pfetable)

    -   [Basic Usage](#basic-usage)
    -   [Keep and drop variables](#keep-and-drop-variables)
    -   [Hide fixed effects or SE-type rows](#hide-fixed-effects-or-se-type-rows)
    -   [Display p-values or confidence intervals](#display-p-values-or-confidence-intervals)
    -   [Significance levels and rounding](#significance-levels-and-rounding)
    -   [Other output formats](#other-output-formats)
    -   [Custom Styling with Great Tables](#custom-styling-with-great-tables)
    -   [Rename variables](#rename-variables)
    -   [Custom model headlines](#custom-model-headlines)
    -   [Further custom model information](#further-custom-model-information)
    -   [Custom table notes](#custom-table-notes)
    -   [Publication-ready LaTex tables](#publication-ready-latex-tables)
    -   [Rendering Tables in Quarto](#rendering-tables-in-quarto)
    
-   [Regression Tables via `Stargazer`](#regression-tables-via-stargazer)


To begin, we load some libraries and fit a set of regression models. 

In [1]:
import numpy as np
import pylatex as pl  # for the latex table; note: not a dependency of pyfixest - needs manual installation
from great_tables import loc, style
from IPython.display import FileLink, display
from stargazer.stargazer import LineLocation, Stargazer

import pyfixest as pf

%load_ext autoreload
%autoreload 2

data = pf.get_data()

fit1 = pf.feols("Y ~ X1 + X2 | f1", data=data)
fit2 = pf.feols("Y ~ X1 + X2 | f1 + f2", data=data)
fit3 = pf.feols("Y ~ X1 *X2 | f1 + f2", data=data)
fit4 = pf.feols("Y2 ~ X1 + X2 | f1", data=data)
fit5 = pf.feols("Y2 ~ X1 + X2 | f1 + f2", data=data)
fit6 = pf.feols("Y2 ~ X1 *X2 | f1 + f2", data=data)

## Regression Tables via `pf.etable()`

### Basic Usage

We can compare all regression models via the pyfixest-internal `pf.etable()` function: 

In [2]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2    coef          X1:X2                                                   
3      fe             f1                       x                       x   
4      fe             f2                       -                       x   
5   stats   Observations                     997                     997   
6   stats      S.E. type                  by: f1                  by: f1   
7   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2     -0.041 <br> (0.081)  
3                       x  
4                       x  
5                     998  
6                  by: f1  
7                   0.172  , _body=<great_tables._gt_data.Body object at 0x000002525220C7D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253AFD450>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253724210>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font

### Keep and drop variables
`etable` allows us to do a few things out of the box. For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match. 

In [3]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep="X1")

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef          X1:X2                                                   
2      fe             f1                       x                       x   
3      fe             f2                       -                       x   
4   stats   Observations                     997                     997   
5   stats      S.E. type                  by: f1                  by: f1   
6   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1      0.011 <br> (0.018)                                                   
2                       x                       x                       x   
3                       x                       -                       x   
4                     997                     998                     998   
5                  by: f1                  by: f1                  by: f1   
6                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.041 <br> (0.081)  
2                       x  
3                       x  
4                     998  
5                  by: f1  
6                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253B21C50>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000252522DED90>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000252522DE1D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue'

We can use the `exact_match` argument to select a specific set of variables: 

In [4]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep=["X1", "X2"], exact_match=True)

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2      fe             f1                       x                       x   
3      fe             f2                       -                       x   
4   stats   Observations                     997                     997   
5   stats      S.E. type                  by: f1                  by: f1   
6   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2                       x                       x                       x   
3                       x                       -                       x   
4                     997                     998                     998   
5                  by: f1                  by: f1                  by: f1   
6                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2                       x  
3                       x  
4                     998  
5                  by: f1  
6                   0.172  , _body=<great_tables._gt_data.Body object at 0x00000252537BBB10>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000252536B5550>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000252536B6DD0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue'

We can also easily **drop** variables via the `drop` argument: 

In [5]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], drop=["X1"])

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
1      fe             f1                       x                       x   
2      fe             f2                       -                       x   
3   stats   Observations                     997                     997   
4   stats      S.E. type                  by: f1                  by: f1   
5   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                      3                      4  \
0  -0.185*** <br> (0.025)  -0.131** <br> (0.042)  -0.118** <br> (0.042)   
1                       x                      x                      x   
2                       x                      -                      x   
3                     997                    998                    998   
4                  by: f1                 by: f1                 by: f1   
5                   0.659                  0.120                  0.172   

                     5  
0  -0.074 <br> (0.104)  
1                    x  
2                    x  
3                  998  
4               by: f1  
5                0.172  , _body=<great_tables._gt_data.Body object at 0x00000252522D3B10>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025252587450>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253B058D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value=

### Hide fixed effects or SE-type rows
We can hide the rows showing the relevant fixed effects and those showing the S.E. type by setting `show_fe=False` and `show_setype=False` (for instance when the set of fixed effects or the estimation method for the std. errors is the same for all models and you want to describe this in the text or table notes rather than displaying it in the table). 

In [6]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], show_fe=False, show_se_type=False)

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2    coef          X1:X2                                                   
3   stats   Observations                     997                     997   
4   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                     997                     998                     998   
4                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2     -0.041 <br> (0.081)  
3                     998  
4                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253B64BD0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253B37110>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000252521A3810>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, catego

### Display p-values or confidence intervals
By default, `pf.etable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals via the `coef_fmt` function argument. 

In [7]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], coef_fmt="b \n (se) \n [p]")

GT(_tbl_data=  level_0        level_1                                    0  \
0    coef             X1  -0.950*** <br> (0.067) <br> [0.000]   
1    coef             X2  -0.174*** <br> (0.018) <br> [0.000]   
2    coef          X1:X2                                        
3      fe             f1                                    x   
4      fe             f2                                    -   
5   stats   Observations                                  997   
6   stats      S.E. type                               by: f1   
7   stats  R<sup>2</sup>                                0.489   

                                     1                                    2  \
0  -0.924*** <br> (0.061) <br> [0.000]  -0.924*** <br> (0.061) <br> [0.000]   
1  -0.174*** <br> (0.015) <br> [0.000]  -0.185*** <br> (0.025) <br> [0.000]   
2                                           0.011 <br> (0.018) <br> [0.565]   
3                                    x                                    x   
4                                    x                                    x   
5                                  997                                  997   
6                               by: f1                               by: f1   
7                                0.659                                0.659   

                                     3                                    4  \
0  -1.267*** <br> (0.174) <br> [0.000]  -1.232*** <br> (0.192) <br> [0.000]   
1   -0.131** <br> (0.042) <br> [0.005]   -0.118** <br> (0.042) <br> [0.008]   
2                                                                             
3                                    x                                    x   
4                                    -                                    x   
5                                  998                                  998   
6                               by: f1                               by: f1   
7                                0.120                                0.172   

                                     5  
0  -1.231*** <br> (0.192) <br> [0.000]  
1     -0.074 <br> (0.104) <br> [0.482]  
2     -0.041 <br> (0.081) <br> [0.618]  
3                                    x  
4                                    x  
5                                  998  
6                               by: f1  
7                                0.172  , _body=<great_tables._gt_data.Body object at 0x00000252524FD790>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253D42250>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error) \

### Significance levels and rounding
Additionally, we can also overwrite the defaults for the reported significance levels and control the rounding of results via the `signif_code` and `digits` function arguments: 

In [8]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits=5)

GT(_tbl_data=  level_0        level_1                           0  \
0    coef             X1  -0.94953*** <br> (0.06652)   
1    coef             X2  -0.17423*** <br> (0.01840)   
2    coef          X1:X2                               
3      fe             f1                           x   
4      fe             f2                           -   
5   stats   Observations                         997   
6   stats      S.E. type                      by: f1   
7   stats  R<sup>2</sup>                     0.48899   

                            1                           2  \
0  -0.92405*** <br> (0.06093)  -0.92417*** <br> (0.06094)   
1  -0.17411*** <br> (0.01461)  -0.18550*** <br> (0.02516)   
2                                  0.01057 <br> (0.01818)   
3                           x                           x   
4                           x                           x   
5                         997                         997   
6                      by: f1                      by: f1   
7                     0.65904                     0.65916   

                            3                           4  \
0  -1.26655*** <br> (0.17359)  -1.23153*** <br> (0.19228)   
1  -0.13056*** <br> (0.04239)  -0.11767*** <br> (0.04152)   
2                                                           
3                           x                           x   
4                           -                           x   
5                         998                         998   
6                      by: f1                      by: f1   
7                     0.12017                     0.17151   

                            5  
0  -1.23100*** <br> (0.19167)  
1     -0.07369 <br> (0.10356)  
2     -0.04082 <br> (0.08093)  
3                           x  
4                           x  
5                         998  
6                      by: f1  
7                     0.17180  , _body=<great_tables._gt_data.Body object at 0x00000252521A3910>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253709FD0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.1, ** p < 0.05, *** p < 0.01. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002525370BDD0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left

### Other output formats
By default, `pf.etable()` returns a GT object (see [the Great Tables package](https://posit-dev.github.io/great-tables/articles/intro.html)), but you can also opt to dataframe, markdown, or latex output via the `type` argument.

In [9]:
# Pandas styler output:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=5,
    coef_fmt="b (se)",
    type="df",
)

,est1,est2,est3,est4,est5,est6
depvar,Y,Y,Y,Y2,Y2,Y2
X1,-0.94953*** (0.06652),-0.92405*** (0.06093),-0.92417*** (0.06094),-1.26655*** (0.17359),-1.23153*** (0.19228),-1.23100*** (0.19167)
X2,-0.17423*** (0.01840),-0.17411*** (0.01461),-0.18550*** (0.02516),-0.13056*** (0.04239),-0.11767*** (0.04152),-0.07369 (0.10356)
X1:X2,,,0.01057 (0.01818),,,-0.04082 (0.08093)
f1,x,x,x,x,x,x
f2,-,x,x,-,x,x
Observations,997,997,997,998,998,998
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1,by: f1
R2,0.48899,0.65904,0.65916,0.12017,0.17151,0.17180


In [10]:
# Markdown output:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=5,
    type="md",
)

index                 est1          est2          est3          est4          est5          est6
------------  ------------  ------------  ------------  ------------  ------------  ------------
depvar                   Y             Y             Y            Y2            Y2            Y2
------------------------------------------------------------------------------------------------
X1            -0.94953***   -0.92405***   -0.92417***   -1.26655***   -1.23153***   -1.23100***
                 (0.06652)     (0.06093)     (0.06094)     (0.17359)     (0.19228)     (0.19167)
X2            -0.17423***   -0.17411***   -0.18550***   -0.13056***   -0.11767***      -0.07369
                 (0.01840)     (0.01461)     (0.02516)     (0.04239)     (0.04152)     (0.10356)
X1:X2                                         0.01057                                  -0.04082
                                             (0.01818)                                 (0.08093)
---------------------------------

To obtain latex output use `format = "tex"`. If you want to save the table as a tex file, you can use the `filename=` argument to specify the respective path where it should be saved. If you want the latex code to be displayed in the notebook, you can use the `print_tex=True` argument.
Etable will use latex packages `booktabs`, `threeparttable` and `makecell` for the table layout, so don't forget to include these packages in your latex document. 
 

In [11]:
# LaTex output (include latex packages booktabs, threeparttable, and makecell in your document):
tab = pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=2,
    type="tex",
    print_tex=True,
)

The following code generates a pdf including the regression table which you can display clicking on the link below the cell:

In [12]:
## Use pylatex to create a tex file with the table


def make_pdf(tab, file):
    "Create a PDF document with tex table."
    doc = pl.Document()
    doc.packages.append(pl.Package("booktabs"))
    doc.packages.append(pl.Package("threeparttable"))
    doc.packages.append(pl.Package("makecell"))

    with (
        doc.create(pl.Section("A PyFixest LateX Table")),
        doc.create(pl.Table(position="htbp")) as table,
    ):
        table.append(pl.NoEscape(tab))

    doc.generate_pdf(file, clean_tex=False)


# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "latexdocs/SampleTableDoc")
display(FileLink("latexdocs/SampleTableDoc.pdf"))

c:\PythonProjects\pyfixest\docs\latexdocs\SampleTableDoc.pdf

### Custom Styling with Great Tables
You can use the rich set of methods offered by [Great Tables](https://posit-dev.github.io/great-tables/articles/intro.html) to further customize the table display.

In [13]:
(
    pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])
    .tab_options(
        column_labels_background_color="cornsilk",
        stub_background_color="whitesmoke",
    )
    .tab_style(
        style=style.fill(color="mistyrose"),
        locations=loc.body(columns="(3)", rows=["X2"]),
    )
)

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2    coef          X1:X2                                                   
3      fe             f1                       x                       x   
4      fe             f2                       -                       x   
5   stats   Observations                     997                     997   
6   stats      S.E. type                  by: f1                  by: f1   
7   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2     -0.041 <br> (0.081)  
3                       x  
4                       x  
5                     998  
6                  by: f1  
7                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253DDF990>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002525220DE10>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253E26110>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font

### Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).

In [14]:
labels = {
    "Y": "Wage",
    "Y2": "Wealth",
    "X1": "Age",
    "X2": "Years of Schooling",
    "f1": "Industry",
    "f2": "Year",
}

pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                  by: f1                  by: f1  
7                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253F56C10>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253DEF090>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253E391D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scs

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.

In [15]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"},
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe         Industry Fixed Effects                       x   
4      fe             Year Fixed Effects                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                  by: f1                  by: f1  
7                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253E39190>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253E5EF50>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253E5C050>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scs

### Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.

In [16]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    model_heads=["US", "China", "EU", "US", "China", "EU"],
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                  by: f1                  by: f1  
7                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253E247D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253EEB250>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=2, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=2, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None), SpannerInfo(spanner_id='US', spanner_level=1, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['0', '3'], built=None), SpannerInfo(spanner_id='China', spanner_level=1, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['1', '4'], built=None), SpannerInfo(spanner_id='EU', spanner_level=1, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['2', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253EE1750>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(sc

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are shown.

In [17]:
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -1.267*** <br> (0.174)  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)   
1   -0.131** <br> (0.042)  -0.174*** <br> (0.015)   -0.118** <br> (0.042)   
2                                                                           
3                       x                       x                       x   
4                       -                       x                       x   
5                     998                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.120                   0.659                   0.172   

                        4                       5  
0  -0.924*** <br> (0.061)  -1.231*** <br> (0.192)  
1  -0.185*** <br> (0.025)     -0.074 <br> (0.104)  
2      0.011 <br> (0.018)     -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     997                     998  
6                  by: f1                  by: f1  
7                   0.659                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253E3D790>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253F104D0>, _spanners=Spanners([SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['0', '1'], built=None), SpannerInfo(spanner_id='China', spanner_level=2, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['2', '3'], built=None), SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['4', '5'], built=None), SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '2', '4'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['1', '3', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253FA29D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(sc

Remove the dependent variables from the headers:

In [18]:
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="",
)

GT(_tbl_data=  level_0                        level_1                     (1)  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                      (2)                     (3)                     (4)  \
0  -1.267*** <br> (0.174)  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)   
1   -0.131** <br> (0.042)  -0.174*** <br> (0.015)   -0.118** <br> (0.042)   
2                                                                           
3                       x                       x                       x   
4                       -                       x                       x   
5                     998                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.120                   0.659                   0.172   

                      (5)                     (6)  
0  -0.924*** <br> (0.061)  -1.231*** <br> (0.192)  
1  -0.185*** <br> (0.025)     -0.074 <br> (0.104)  
2      0.011 <br> (0.018)     -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     997                     998  
6                  by: f1                  by: f1  
7                   0.659                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253EE9C50>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253E41850>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000025253E43DD0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Rob

### Further custom model information
You can add further custom model statistics/information to the bottom of the table by using the `custom_stats` argument to which you pass a dictionary with the name of the row and lists of values. The length of the lists must be equal to the number of models.

In [19]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
        "Additional Info": ["A", "A", "B", "B", "C", "C"],
    },
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats             Number of Clusters                      42   
6   stats                Additional Info                       A   
7   stats                   Observations                     997   
8   stats                      S.E. type                  by: f1   
9   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                      42                      42                      37   
6                       A                       B                       B   
7                     997                     997                     998   
8                  by: f1                  by: f1                  by: f1   
9                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                      37                      37  
6                       C                       C  
7                     998                     998  
8                  by: f1                  by: f1  
9                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x000002525371C7D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000025253F4E650>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002525413FE50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table'

### Custom table notes
You can replace the default table notes with your own notes using the `notes` argument. 


In [20]:
mynotes = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    notes=mynotes,
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -1.267*** <br> (0.174)  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)   
1   -0.131** <br> (0.042)  -0.174*** <br> (0.015)   -0.118** <br> (0.042)   
2                                                                           
3                       x                       x                       x   
4                       -                       x                       x   
5                     998                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.120                   0.659                   0.172   

                        4                       5  
0  -0.924*** <br> (0.061)  -1.231*** <br> (0.192)  
1  -0.185*** <br> (0.025)     -0.074 <br> (0.104)  
2      0.011 <br> (0.018)     -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     997                     998  
6                  by: f1                  by: f1  
7                   0.659                   0.172  , _body=<great_tables._gt_data.Body object at 0x0000025253E3E6D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002525400E990>, _spanners=Spanners([SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['0', '1'], built=None), SpannerInfo(spanner_id='China', spanner_level=2, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['2', '3'], built=None), SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['4', '5'], built=None), SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '2', '4'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['1', '3', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit

### Publication-ready LaTex tables 
With few lines of code you thus obtain a publication-ready latex table:

In [21]:
tab = pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    type="tex",
    notes=mynotes,
    show_fe=True,
    show_se_type=False,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
    },
)

# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "latexdocs/SampleTableDoc2")
display(FileLink("latexdocs/SampleTableDoc2.pdf"))

c:\PythonProjects\pyfixest\docs\latexdocs\SampleTableDoc2.pdf

### Rendering Tables in Quarto
When you use quarto you can include latex tables generated by `pf.etable` when rendering the qmd file as pdf. Just specify `output: asis` in the code block options of the respective chunk and print the LaTex string returned by etable. Don't forget to include the `\usepackage` commands for necessary latex packages in the YAML block.

Here you find a sample [qmd file](https://github.com/py-econometrics/pyfixest/blob/master/docs/quarto_example/QuartoExample.qmd) and the rendered [PDF](quarto_example/QuartoExample.pdf).

## Regression Tables via `Stargazer`

We have opened a PR for `pyfixest` support for the excellent [Stargazer](https://github.com/StatsReporting/stargazer/pull/105) project. Until it is merged, you can download the dev version from `py-econometrics` by typing

```bash
pip install git+https://github.com/py-econometrics/stargazer.git
```

`Stargazer` is particularly useful if you need highly customizable regression tables (beyond the scope of `pf.etable()`), or if you want to compare models from `statsmodels` or `linearmodels` with `pyfixest`. 

After installing `stargazer`, we can produce a summary table via the `Stargazer` class: 

In [22]:
stargazer_table = Stargazer([fit1, fit2, fit3, fit4, fit5, fit6])
stargazer_table

NotImplementedError: <class 'pyfixest.estimation.feols_.Feols'>

We can easily add custom statisics. For example, assume that we want to correct for multiple testing via the Romano-Wolf correction. We can do this as follows:

In [ ]:
rwolf_res = pf.rwolf(
    [fit1, fit2, fit3, fit4, fit5, fit6], param="X1", seed=123, reps=9999
)
rwolf_pvalues = np.round(rwolf_res.xs("RW Pr(>|t|)"), 3).to_list()

: 

In [ ]:
stargazer_table.add_line(
    "Fixed Effects",
    [x._fixef for x in [fit1, fit2, fit3, fit4, fit5, fit6]],
    LineLocation.FOOTER_TOP,
)
stargazer_table.add_line(
    "X1: Romano-Wolf P-Value", rwolf_pvalues, LineLocation.FOOTER_TOP
)
stargazer_table

: 